In [ ]:
#ipython magic
%reset -f
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append(r'C:\Users\Robert Lees\Documents\Code\Vape\suite2p_etc')
sys.path.append('..')

import numpy as np
import os
import time

from utils.gsheets_importer import gsheet2df, correct_behaviour_df, split_df, df_bool, df_col, path_conversion
from utils.utils_funcs import *
from utils.paq2py import *
import tifffile

import matplotlib.pyplot as plt

import suite2p
print(suite2p.__path__)
from suite2p.run_s2p import run_s2p
from settings import ops

import matplotlib.pyplot as plt

In [ ]:
# spreadsheet settings

#the name of the file path column
t_series_header = 'Path (file or folder)'
paq_header = '.paq file'
naparm_header = 'NAPARM path'

#column telling whether to analyse or not
suite2p_bool_header = 'Suite2p_me'

In [ ]:
sheet_ID = '1PgSXs6BVHe9dACWEydykucN74rirNfOY4Vr_AmLcCdY'
mouse_name = 'RL042'
date = '2019-05-15_'
SHEET_NAME = date + mouse_name + '!A3:Z69'

df = gsheet2df(sheet_ID, HEADER_ROW=2, SHEET_NAME=SHEET_NAME)
df

# RL in future we should change Suite2p_me to FALSE once it has been run

In [ ]:
analyse_idx = df.index[df[suite2p_bool_header]=='TRUE']
tseries_paths = [df.loc[idx,t_series_header] for idx in analyse_idx]
paq_paths = [df.loc[idx,paq_header] for idx in analyse_idx]
sampling_rates = [df.loc[idx,'~FPS'] for idx in analyse_idx]
zoom = [df.loc[idx,'Zoom'] for idx in analyse_idx]
# naparm_paths = [df.loc[idx,naparm_header] for idx in analyse_idx]

print(tseries_paths)
print(paq_paths)
print(sampling_rates)
print(zoom)

In [ ]:
packerstation_path = r"P:"

tseries_pstation = path_conversion(tseries_paths, packerstation_path)
paqs_pstation = path_conversion(paq_paths, packerstation_path)
# naparms_pstation = path_conversion(naparm_paths)

print(tseries_pstation)
print(paqs_pstation)

In [ ]:
#set paths to tiff here, any settings set here will overwrite ops settings, meaning we can set
#different cell sizes with different zooms here.
#JR added parameter has_stim = whether to remove photostim artifact

#important: data paths must be lists even if only one element

# only run on specified tiffs
db = []

for i,folder_name in enumerate(tseries_pstation):

#     tiff_list = []
    
#     for file in os.listdir(folder_name):
#             if '.ome' not in file and '.tif' in file:
#                 tiff_list.append(file) 
    
#     tiff_list = sorted(tiff_list)
    
    if float(zoom[i]) < 2:
        diameter = 10
        batch_size = 200 # this should be eventually decided based on image size (in pixels) as a proxy of file size, also depends on CPU capacity
        
    if float(zoom[i]) == 2:
        diameter = 12
        batch_size = 500
        
#     db.append( { 'data_path' : ['P:\\\\sarmstrong\\Data\\2019-05-15\\RL042\\2019-05-15_t-003'],
#            'has_stim' : False} )

    db.append({ 'data_path' : [folder_name], 
#               'tiff_list' : tiff_list,
              'fs' : int(sampling_rates[i])/3,
              'diameter' : diameter, 
              'batch_size' : batch_size, 
              'nimg_init' : batch_size
              })
    
db

In [ ]:
t1 = time.time()

for dbi in db:
#     if dbi['has_stim']:
#         dbi = process_stim_artefact(db=dbi,threshold=1.3, interpolate=False)
    opsEnd = run_s2p(ops=ops,db=dbi)
    
t2 = time.time()
t2 - t1